# __Poisson matrix tests__

## Outline
- __Dependencies__
- __Dynamic ansatz__
    - Noiseless
    - With noise using Qiskit AER
- __Static ansatz__
    - Noiseless
    - With noise using Qiskit AER

In [4]:
import numpy as np
from device_var_lse_solver import DeviceVarLSESolver
from device import DeviceType
from conditioned_matrix import poissonMatrix
from non_dynamical_ansatz import fixed_layered_ansatz
from tqdm import tqdm
from device import Device

qubits = 4
b = np.ones(2**qubits)/np.sqrt(2**qubits)
depth=10

assert qubits % 2 == 0 # Needed for Poisson matrix

In [5]:
# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

## __Dynamic ansatz__

In [6]:
# For poisson matrix
N_runs = 10
N_steps = 1500
redo_calc = 5

local_noiseless_dynamic_trc = np.zeros((redo_calc, ))
global_noiseless_dynamic_trc = np.zeros((redo_calc, ))
local_noise_dynamic_trc = np.zeros((redo_calc, ))
global_noise_dynamic_trc = np.zeros((redo_calc, ))

local_noiseless_static_trc = np.zeros((redo_calc, ))
global_noiseless_static_trc = np.zeros((redo_calc, ))
local_noise_static_trc = np.zeros((redo_calc, ))
global_noise_static_trc = np.zeros((redo_calc, ))

local_noise_dynamic_error = np.zeros((redo_calc, ))
local_noiseless_dynamic_error = np.zeros((redo_calc, ))
local_noise_static_error = np.zeros((redo_calc, ))
local_noiseless_static_error = np.zeros((redo_calc, ))

global_noise_dynamic_error = np.zeros((redo_calc, ))
global_noiseless_dynamic_error = np.zeros((redo_calc, ))
global_noise_static_error = np.zeros((redo_calc, ))
global_noiseless_static_error = np.zeros((redo_calc, ))

local_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noise_static_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

global_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noise_static_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

correct_solutions = np.zeros((redo_calc, 2**qubits))


device = Device(DeviceType.QISKIT_AER, qubits=qubits)

param_shape = (qubits + depth*(qubits+qubits -2),)

pbar = tqdm(range(redo_calc))
for i in pbar:
        a = poissonMatrix(2**(qubits//2)).todense()
        classical_solution = np.linalg.solve(a, b)
        normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))
        normalized_classical_solution /= np.linalg.norm(normalized_classical_solution)

        correct_solutions[i] = np.abs(normalized_classical_solution)

        ###################################
        #####    DYNAMIC NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=False)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        pbar.set_description("Local Dynamic Noiseless (1/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Dynamic Noiseless (2/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_dynamic_solution[i] = solution_local
        global_noiseless_dynamic_solution[i] = solution_global

        local_noiseless_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noiseless_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noiseless_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    DYNAMIC NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        pbar.set_description("Local Dynamic Noise (3/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Dynamic Noise (4/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_dynamic_solution[i] = solution_local
        global_noise_dynamic_solution[i] = solution_global

        local_noise_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noise_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noise_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        pbar.set_description("Local Static Noiseless (5/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Static Noiseless (6/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_static_solution[i] = solution_local
        global_noiseless_static_solution[i] = solution_global

        local_noiseless_static_trc[i] = TRC_ASA(it_count_local)
        global_noiseless_static_trc[i] = TRC_ASA(it_count_global)

        local_noiseless_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        pbar.set_description("Local Static Noise (7/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Static Noise (8/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_static_solution[i] = solution_local
        global_noise_static_solution[i] = solution_global

        local_noise_static_trc[i] = TRC_ASA(it_count_local)
        global_noise_static_trc[i] = TRC_ASA(it_count_global)

        local_noise_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

Local Dynamic Noiseless (1/8):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/10:  39%|███▊      | 580/1500 [00:08<00:14, 65.26it/s, best loss=0.167, last improvement in step=82, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  41%|████      | 614/1500 [00:17<00:25, 34.85it/s, best loss=0.118, last improvement in step=116, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  39%|███▉      | 589/1500 [00:31<00:48, 18.92it/s, best loss=0.118, last improvement in step=89, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  50%|████▉     | 743/1500 [00:51<00:52, 14.51it/s, best loss=0.0638, last improvement in step=244, loss=0.0639]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  40%|████      | 606/1500 [00:50<01:15, 11.89it/s, best loss=0.0638, last improvement in step=107, loss=0.0643]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  40%|████      | 600/1500 [01:02<01:33,  

Global Dynamic Noise (4/8):   0%|          | 0/5 [31:41<?, ?it/s]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  20%|██        | 1/5 [44:44<2:58:59, 2684.81s/it]

Epoch 1/10:  39%|███▉      | 591/1500 [00:09<00:15, 60.11it/s, best loss=0.167, last improvement in step=97, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  40%|████      | 600/1500 [00:16<00:24, 36.06it/s, best loss=0.118, last improvement in step=103, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  49%|████▊     | 729/1500 [00:28<00:30, 25.63it/s, best loss=0.0637, last improvement in step=231, loss=0.0638]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  47%|████▋     | 710/1500 [00:35<00:39, 19.95it/s, best loss=0.0637, last improvement in step=210, loss=0.065]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  50%|████▉     | 744/1500 [00:46<00:47, 16.00it/s, best loss=0.0637, last improvement in step=245, loss=0.0642]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  44%|████▍     | 664/1500 [00:48<01:01,

Global Dynamic Noise (4/8):  20%|██        | 1/5 [1:06:49<2:58:59, 2684.81s/it]  c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  40%|████      | 2/5 [1:19:02<1:55:47, 2315.98s/it]

...

Global Dynamic Noise (4/8):  40%|████      | 2/5 [1:47:42<1:55:47, 2315.98s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  60%|██████    | 3/5 [2:02:07<1:21:17, 2438.67s/it]

Epoch 1/10:  39%|███▊      | 578/1500 [00:10<00:16, 55.09it/s, best loss=0.167, last improvement in step=80, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  40%|███▉      | 594/1500 [00:20<00:31, 28.76it/s, best loss=0.118, last improvement in step=97, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  50%|█████     | 753/1500 [00:36<00:36, 20.65it/s, best loss=0.0638, last improvement in step=255, loss=0.0637]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  47%|████▋     | 704/1500 [00:45<00:51, 15.59it/s, best loss=0.0638, last improvement in step=204, loss=0.0641]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  43%|████▎     | 652/1500 [00:52<01:08, 12.47it/s, best loss=0.0638, last improvement in step=152, loss=0.0645]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  45%|████▍     | 668/1500 [01:03<01:18,

Global Dynamic Noise (4/8):  60%|██████    | 3/5 [2:27:39<1:21:17, 2438.67s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  80%|████████  | 4/5 [2:40:12<39:37, 2377.91s/it]  

Epoch 1/10:  38%|███▊      | 574/1500 [00:10<00:16, 54.60it/s, best loss=0.167, last improvement in step=76, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  40%|███▉      | 597/1500 [00:19<00:30, 29.98it/s, best loss=0.118, last improvement in step=98, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  46%|████▋     | 697/1500 [00:33<00:38, 20.70it/s, best loss=0.0638, last improvement in step=199, loss=0.0639]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  48%|████▊     | 724/1500 [00:46<00:49, 15.70it/s, best loss=0.0638, last improvement in step=224, loss=0.0647]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  60%|█████▉    | 898/1500 [01:09<00:46, 12.89it/s, best loss=0.0638, last improvement in step=399, loss=0.065]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  42%|████▏     | 627/1500 [00:59<01:22, 

Global Dynamic Noise (4/8):  80%|████████  | 4/5 [3:04:37<39:37, 2377.91s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Global Static Noise (8/8): 100%|██████████| 5/5 [3:16:47<00:00, 2361.49s/it]    


## __Save vals__

In [ ]:
## Save the vals
filename = f"data/result_poisson_test_2"
saved_dict = {
    # Noise dynamic
    "local_noise_dynamic_trc" : local_noise_dynamic_trc,
    "local_noise_dynamic_error" : local_noise_dynamic_error,
    "local_noise_dynamic_solution" : local_noise_dynamic_solution,
    "global_noise_dynamic_trc" : global_noise_dynamic_trc,
    "global_noise_dynamic_error" : global_noise_dynamic_error,
    "global_noise_dynamic_solution" : global_noise_dynamic_solution,

    # Noiseless dynamic
    "local_noiseless_dynamic_trc" : local_noiseless_dynamic_trc,
    "local_noiseless_dynamic_error" : local_noiseless_dynamic_error,
    "local_noiseless_dynamic_solution" : local_noiseless_dynamic_solution,
    "global_noiseless_dynamic_trc" : global_noiseless_dynamic_trc,
    "global_noiseless_dynamic_error" : global_noiseless_dynamic_error,
    "global_noiseless_dynamic_solution" : global_noiseless_dynamic_solution,

    # Noise static
    "local_noise_static_trc" : local_noise_static_trc,
    "local_noise_static_error" : local_noise_static_error,
    "local_noise_static_solution" : local_noise_static_solution,
    "global_noise_static_trc" : global_noise_static_trc,
    "global_noise_static_error" : global_noise_static_error,
    "global_noise_static_solution" : global_noise_static_solution,

    # Noiseless static
    "local_noiseless_static_trc" : local_noiseless_static_trc,
    "local_noiseless_static_error" : local_noiseless_static_error,
    "local_noiseless_static_solution" : local_noiseless_static_solution,
    "global_noiseless_static_trc" : global_noiseless_static_trc,
    "global_noiseless_static_error" : global_noiseless_static_error,
    "global_noiseless_static_solution" : global_noiseless_static_solution,

    "normalized_classical_solution" : normalized_classical_solution
}
np.savez(filename, **saved_dict)
print(f"Saved {filename}")

Saved data/result_poisson_test_1


In [8]:
data = np.load(f"{filename}.npz")

local_noise_dynamic_error = data["local_noise_dynamic_error"]
print(local_noise_dynamic_error)

[0.26723437 0.24467666 0.24696304 0.29378532 0.31742005]
